In [7]:
import fitz
import re
import os
from PIL import Image
from tkinter import filedialog
import os
from glob import glob
from tqdm import tqdm

def pdf2pic(path, pic_path):
    checkXO = r"/Type(?= */XObject)"  # 使用正则表达式来查找图片
    checkIM = r"/Subtype(?= */Image)"
    doc = fitz.open(path)  # 打开pdf文件
    imgcount = 0  # 图片计数
    lenXREF = doc.xref_length()    # 获取对象数量长度
    pic_path_d_list = []

    # 打印PDF的信息
    print("文件名:{}, 页数: {}, 对象: {}".format(path, len(doc), lenXREF - 1))

    # 遍历每一个对象
    for i in range(1, lenXREF):
        text = doc.xref_object(i)  # 定义对象字符串
        isXObject = re.search(checkXO, text)  # 使用正则表达式查看是否是对象
        isImage = re.search(checkIM, text)  # 使用正则表达式查看是否是图片
        if not isXObject or not isImage:  # 如果不是对象也不是图片，则continue
            continue
        imgcount += 1
        pix = fitz.Pixmap(doc, i)  # 生成图像对象
        
        new_name =path.split('/')[-1]  + "_{}.jpg".format(imgcount)  # 生成图片的名称
        print(new_name)
        if pix.n < 5:  # 如果pix.n<5,可以直接存为PNG
            pic_path_d = os.path.join(pic_path, 'pic',new_name)
            pix.writeImage(pic_path_d)
            im = Image.open(pic_path_d)
            x, y = im.size
            if x > 1376:
                y_s = int(y * 1376 / x)
                im = im.resize((1376, y_s), Image.ANTIALIAS)
            im.save(pic_path_d)
            pic_path_d_list.append(pic_path_d)

        else:  # 否则先转换CMYK
            pix0 = fitz.Pixmap(fitz.csRGB, pix)
            pix0.writeImage(pic_path_d)
            pix0 = None

        pix = None  # 释放资源
#     print("提取了{}张图片".format(imgcount))
    
    doc = fitz.open()
    # 循环path中的文件，可import os 然后用 for img in os.listdir(img_path)实现
    # 这里为了让文件以1，2，3的形式进行拼接，就偷懒循环文件名中的数字。
    for img_file in pic_path_d_list:
        imgdoc = fitz.open(img_file)
        pdfbytes = imgdoc.convertToPDF()
#         pdf_name = str(i) + '.pdf'
        pdf_name = os.path.join(pic_path, path.split('/')[-1])
        imgpdf = fitz.open(pdf_name, pdfbytes)
        doc.insertPDF(imgpdf)
    print(pdf_name)
    doc.save(pdf_name)
    doc.close()    
        
def find_pdf(path="."):
    pdf_list = []
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            pdf_list.append(os.path.join(root, name))
    #     for name in dirs:
    #         pdf_list.append(os.path.join(root, name))
    return list(filter(lambda x:x.endswith('pdf'),pdf_list))

In [8]:
def find_pdf(path="."):
    pdf_list = []
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            pdf_list.append(os.path.join(root, name))
    #     for name in dirs:
    #         pdf_list.append(os.path.join(root, name))
    return list(filter(lambda x:x.endswith('pdf'),pdf_list))

In [10]:
save_dir = './compress'
pdf_dir = './剧本'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)
for i in tqdm(find_pdf('./剧本')):
    print(1)
    break
    pdf2pic(i,'./compress')

  0%|          | 0/14 [00:00<?, ?it/s]

1
